> # **Group Prediction with Meta-Estimators**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklego.datasets import load_chicken
from sklego.preprocessing import ColumnSelector

In [ ]:
df = load_chicken(as_frame=True)
df.sample(10)

In [ ]:
def plot_model(model):
    df = load_chicken (as_frame=True)
    model.fit(df[['diet', 'time']], df['weight'])
    metric_df = df[['diet', 'time', 'weight']].assign(pred=lambda df: model.predict(df[['diet', 'time']]))
    print(metric_df)
    print (metric_df.shape)
    metric = mean_absolute_error(metric_df['weight'], metric_df['pred'])
    plt.figure(figsize=(12, 4))
    # plt.scatter(df['time'], df['weight'])
    for i in [1, 2, 3, 4]:
        pltr = metric_df[['time', 'diet', 'pred']].drop_duplicates().loc[lambda df: df['diet'] == i]
        print (pltr.head())
        print (pltr.shape)
        plt.plot(pltr['time'], pltr['pred'], color='.rbgy' [i])
    plt.title(f'linear model per group, MAE: {np.round(metric, 2)}');

In [ ]:
feature_pipeline = Pipeline([
    ("datagrab", FeatureUnion([
        ("discrete", Pipeline([
            ("grab", ColumnSelector("diet")),
            ("encode", OneHotEncoder())
        ])),
        ("continuous", Pipeline([
            ("grab", ColumnSelector("time")),
            ("standardize", StandardScaler())
        ]))
    ]))
])

pipe = Pipeline([
    ("transform", feature_pipeline),
    ("model", LinearRegression())])

plot_model(pipe)

### *We also use this code instead of feature_pipeline (Better MAE)*

In [ ]:
from sklego.meta import GroupedPredictor
mod = GroupedPredictor (LinearRegression(), groups=["diet"])
plot_model(mod)